# Summary of Project

The following notebook runs through the general methodology as well as the key findings across the project, this also contains visual aids in form of graphs, screenshots of notebook activities, tables and cell blocks. Our aim was to develop a model that can predict future rental prices to effectively deliver insights on rental properties in the suburbs of Victoria for consulting an online real estate company.

The Sections of this notebook are split into the 6 parts of our project pipeline:
1. Data Gathering
2. Preprocessing
3. Preliminary Analysis
4. Modelling
5. Key Findings
6. Conclusion

Each section includes a summary of the overall approach, limitations encountered and assumptions made.

## 1. Data Gathering

### 1.1 Our Approach
The first stage of the project was to gather as much relevant data as possible. This was done through sourcing as much publicly avaliable data on the internet that we could find. 

In the end a total of 20 datasets were gathered, these are all listed in the readme file, 17 of which were used in our final analysis and modelling. Some datasets, such as crime and SAL ABS data, were identified as valuable early in the planning stage, while others, like coastline data, proved useful after preliminary analysis. Three datasets — Domain, SA2 ABS data and internet speed data — were ultimately excluded from the analysis, as they could not be effectively preprocessed. We discuss the challenges of these datasets below.

The Domain rental data consistently showed higher rental prices compared to the DFFH data. This discrepancy may be due to Domain focusing primarily on higher-end rental properties or more active rental listings, which could skew prices upwards compared to DFFH, and might include a broader range of rental properties. Due to this inconsistency, we decided not to use the Domain data and instead relied on Department of Families, Fairness, and Housing (DFFH) rental data.

The boundaries of the SA2s (Statistical Area 2) of the SA2 ABS (Australian Bureau of Statistics) data did not line up well with the suburb boundaries of the DFFH data at all, ABS data of SA2s nature had to be disregarded due to the mismatch border sizes as the data would thus not be reflective of the DFFH suburb. This was a setback as parts of preprocessing had already been done converting other external data being transformed to represent a SA2 level. Fortunately the ABS had the same data available in the granularity of SAL (Suburb and Localities) which aligned better with the DFFH data. This did present it's own challenges as discussed in the next section but allowed for ABS data to be included, it just required some extra time and manpower to retrieve this data at SAL level and convert the already preprocess data from an SA2 level to SAL.

The Speedtest data, although initially considered, only covered up to 2019, which did not align with our project’s data range extending from 2016 to 2024. This limitation meant it could not be effectively integrated into our analysis.

The most significant challenge during this stage was the lack of historical rental data on domain.com.au and the lack of permission to scrape realestate.com.au or oldlistings.com.au. We had initially hoped to scrape individual property data and aggregate it at the suburb level for better accuracy. However, due to these constraints, we relied on rental data provided by the DFFH. While this served as a reasonable workaround, the inability to obtain more granular data may have had an impact on the final accuracy of our predictions. Another challenge included daily API call limits for proximity data. A higher quota would have allowed for more varied analysis, potentially resulting in additional insights.

### 1.2 Summary of Challenges, Limitations & Assumptions for this Stage
Challenges
- Sourcing historical rent data
- Finding a granularity that best aligned with the DFFH suburb data

Limitations
- Domain rental data was inconsistent with the DFFH rental data and was thus not sufficient for analysis
- Internet speed test data was additionally not sufficient for analysis due to a lack of data on 2016-2018
- API call limits

Assumptions
- All data gathered was valid and true

## 2. Preprocessing

### 2.1 Our Approach
In the preprocessing stage of our project, we focused on Victorian suburbs, aligning with the definitions and mappings provided by the Australian Bureau of Statistics (ABS) for suburbs and localities, transforming all data so to represent this granularity if it did not already. However, a challenge arose with data from the Department of Families, Fairness, and Housing (DFFH), which utilized unique suburb clusters, clustering suburbs with similar rental prices rather than individual suburbs. To address this, we had to manually map each suburb within these clusters to derive our comprehensive list of suburbs. Despite reaching out to the DFFH for a shapefile that could clarify the exact composition of these clusters, we did not receive a response, so we proceeded by using ABS-defined suburb (SAL) maps to carry out the mapping process.

Through splitting these clusters we were able to increase the number of training samples for modelling although this meant suburbs in the same suburb cluster would have the same rent, this was a safe assumption to make as these suburbs were clustered together for that very reason.

<img src='plots/Summary_Suburb_Clusters_DFFH.png' alt="My Image" width="600"/>


Each individual dataset required it's own unique preprocessing before it could be combined. This was due to each dataset being stored in different forms and sizes and recorded over different time periods. Specifically how each dataset is preprocessed is outlined through comments in the notebooks of the preprocessing folders. However most followed the general preprocessing procedure outlined below:
1. Read in the data
2. Clean the data
3. Transform to the data to represent an SAL level if it was not already
4. Find SAL code for each data point so that the data can be merged on this
5. ARIMA for future data that is not stationary and fill in missing values with the average

Please note that some preprocessing was done after preliminary analysis where final feature engineering was done after findings in the analysis.

In order to forecast future rent data, we had to forecast the data of predictor variables that changes through time such as crime rates and business listings. Where an expert forecast could not be sourced from the internet we used ARIMA to forecast this. Despite this model assuming linearity in the data and relies heavily on historical trends where assumptions that may not always hold true in the dynamic real estate market. We still assumed that ARIMA would be an appropriate forecasting method

Another noteable assumption made was on the DFFH rental data. This was recorded as the median quarterly weekly rent for each year. Since most of our data was recorded annually rather than quarterly, we took the average of these quarterly recorded data for each year and this average of quarterly medians was assumed to be the average weekly rent for that year. As the average of the quarterly medians should approximately be equal to the annual average weekly rent for a given year.

For data involving average distance or time calculations, we used the centroid (the geographical center) of each suburb. This decision was made to prevent any data privacy concerns, as using the actual coordinates of an individual's home without consent would not have been ethical.

To combine all the data, we performed left joins based on the suburbs to ensure consistency across the dataset. Through this we were able to create our final dataset compiled from 17 distinct data sources, covering the years from 2016 to 2024, and it included 566 Victorian suburbs. We identified 36 unique features relevant to predicting rental prices, such as median school scores and Airbnb counts. 

### 2.2 Summary of Challenges, Limitations & Assumptions for this Stage
Challenges
- Manually mapping DFFH suburb clusters to SAL suburb boundaries
- Each dataset usually required unique preprocessing methods that had to be adapted to the dataset specifically, no specific repetitive method could be applied for preprocessing, this was difficult due to the number of our sourced datasets.

Limitations
- Data of predictor variables that changes through time had to be forecasted
- Calculating time and distance proximity measures of suburbs whilst retaining data privacy

Assumptions
- ARIMA would be an appropriate forecasting method for predictor variables data that change through time
- If a data point was missing in a column of a dataset the average of that col was taken to be the value of the missing data point
- average of quarterly recorded DFFH rental data for the year was taken to be reflective of the annual average weekly rent
- SALs in the same DFFH suburb cluster had approximately the same average weekly rent
- Geographical suburb centre served as the average endpoint for proximity calculations to the beach, train stations and CBD, (done for data privacy reasons).

## 3. Preliminary Analysis

### 3.1 Our Approach
Our preliminary analysis involved exploring the relationships our features had with each other, particularly relationships with rent mainly using various different data visualisations between variables. We uncovered key insights that significantly shaped our modeling approach. 

One of the most important discoveries was the impact of inflation and average household size on rental prices. Inflation was found to influence rent independently of a house’s intrinsic value, reflecting the changing value of the dollar over time. Additionally, household size had a notable effect on property prices, but this was not directly correlated with the underlying value of properties within a suburb. To account for these factors, we adjusted our target variable to estimate inflation-adjusted rent per person, which resulted in a much cleaner distribution for making accurate predictions over time, as demonstrated by the graphs. Thus we transformed the response variable—median rental price—by adjusting for inflation and people per household.

<div style="display: flex; justify-content: space-between;">

  <!-- First Image -->
  <div style="width: 45%;">
    <img src="plots/prel1.png" alt="First Image" style="width: 100%; height: auto;">
  </div>

  <!-- Second Image -->
  <div style="width: 45%;">
    <img src="plots/prel2.png" alt="Second Image" style="width: 100%; height:auto">
  </div>

</div>


Further feature engineering was conducted to refine our model, including converting crime statistics into a per-person metric to account for varying population sizes across suburbs and all predictor variables were logged transformed to create better seperation among predictors. We also discarded suburbs with populations lower than 800, using the elbow method to determine this threshold. Ravenhall was specifically removed due to the majority of its population being prison inmates, which distorted the data and this reduced our number of suburbs we had data on from 566 to 469.

After these transformations, significant patterns began to emerge. For instance, commercial density showed a strong positive correlation with rental prices, indicating that more commercial buildings in an area were linked to higher rents. Similarly, proximity to the CBD displayed a clear pattern, with closer suburbs experiencing higher rents. These insights, formed through detailed feature analysis, laid the foundation for constructing an extremely accurate predictive model for our stakeholders.

<div style="display: flex; justify-content: space-between;">

  <!-- First Image -->
  <div style="width: 45%;">
    <img src="plots/cbd1.png" alt="First Image" style="width: 100%; height:auto">
  </div>

  <!-- Second Image -->
  <div style="width: 45%;">
    <img src="plots/cbd2.png" alt="Second Image" style="width: 100%; height:auto">
  </div>

</div>


### 3.2 Summary of Challenges, Limitations & Assumptions for this Stage
Challenges
- Deciding what to do with suburbs with a low population

Limitations
- No limiations were encountered at this stage

Assumptions
- Suburbs with a population of less than 800 were deemed as outliers and therefore removed

## 4. Modelling

### 4.1 Our Approach
We focused on three main models for our analysis: Linear Regression (LR), Random Forest Regression (RFR), and XGBoost. Linear Regression served as a simple and interpretable baseline, relying on the assumption of basic linear relationships to gauge the predictive capability. We then moved to Random Forest, which was chosen for its robustness and ability to capture more complex trends beyond linear relationships. Finally, we implemented XGBoost, a model known for its efficiency and accuracy, to enhance prediction performance.

To evaluate these models, we applied time series analysis by splitting the data into training and testing sets, ensuring that only past data was used to simulate real-world forecasting. We measured the performance using R-squared (R²) and Root Mean Squared Error (RMSE) on average weekly rent (not our response variable average weekly/(inflation x household size) as weekly rent is our feature of interest). XGBoost emerged as the best-performing model, with an RMSE of 32.5, indicating that the forecasted rent was approximately off by only \$32.5 on average per suburb. Considering the average rental price of a house in Victoria in this test set (2024) is around \$500, the error is approximately 6% of the true value. This is a very good result.

<div style="display: flex; justify-content: space-between;">

  <!-- Image on the left side -->
  <div style="width: 45%;">
    <img src="plots/eval.png" alt="Your Image Description" style="width: 100%;">
  </div>

  <!-- Table on the right side -->
  <div style="width: 45%;">
    <h3>Model (trained 2017-2023, tested 2024)</h3>
    <table>
      <tr>
        <th>Model</th>
        <th>RMSE</th>
        <th>R<sup>2</sup></th>
        <th>Performance</th>
      </tr>
      <tr>
        <td><span style="color:blue;">&#9679;</span> Linear Regression (LR)</td>
        <td>55.9836</td>
        <td>0.3493</td>
        <td><span style="color:orange;">SUB PAR</span></td>
      </tr>
      <tr>
        <td><span style="color:green;">&#9679;</span> Random Forest (RF)</td>
        <td>35.7335</td>
        <td>0.7349</td>
        <td><span style="color:green;">VERY GOOD</span></td>
      </tr>
      <tr>
        <td><span style="color:orange;">&#9679;</span> XGBoost (XG)</td>
        <td>32.5025</td>
        <td>0.7807</td>
        <td><span style="color:green;">EXCELLENT</span></td>
      </tr>
    </table>
  </div>

</div>


 The comparison graph highlights the strong performance of all models over time, with an upward trend in rental prices. Using ARIMA for future estimates of our predictor variables, we extended our predictions from 2024 to 2028, showing a continued rise in rent prices. Linear Regression, our baseline model, did not perform very well when there was a lack of data (2017 and 2018) and struggled model the effects of COVID well. However this emphasised how robust our models Random Forest and XGBoost were through out modelling, even capturing irregular events like the impact of COVID-19!

<img src='plots/model_strength.png' alt="My Image" width="800"/>


### 4.2 Summary of Challenges, Limitations & Assumptions for this Stage
Challenges
- Modelling COVID-19 with Linear Regression consistently well

Limitations
- Only able to access publicly avaliable
- Specific Model Weakenesses

Assumptions
- No assumptions were made at this stage of the pipeline

## 5. Key Findings

#### 5.1 'What are the most important internal and external features in predicting rental prices?'

Feature importance analysis reveals that both internal and external factors play a crucial role in determining rental prices. Internal factors, such as proximity to the CBD, reflect aspects intrinsic to the property, while external factors, like food establishment density, relate to the surrounding environment. Understanding how these features interact allows our model to make more accurate predictions based on a comprehensive assessment of what influences land value.
The graph visually reinforces this, highlighting the most influential predictors in our XGBoost and Random Forest models. We can clearly see that factors like distance to the CBD and proximity to the beach top the list, showcasing the importance of location-based variables. The combination of these insights ensures our model can deliver precise rental price predictions, making it highly valuable for stakeholders seeking data-driven decision-making tools.


<img src='plots/feature_importance_combined.png' alt="My Image" width="800"/>


#### 5.2 What are the top 10 suburbs with the highest predicted growth rate?

Our model has demonstrated good accuracy in predicting historical growth from 2019-2023, having almost perfectly identified the top ten growth suburbs. 

### Suburb Growth Comparison 2019-2023:

| Rank | Suburb        | Actual Growth | Predicted Growth (Predicted Rank) |
|------|---------------|---------------|-----------------------------------|
| 1.   | Sebastopol    | 74.50%        | 73.46% (1)                        |
| 2.   | Wodonga       | 64.67%        | 65.48% (2)                        |
| 3.   | Moe           | 63.29%        | 58.88% (4)                        |
| 4.   | Newborough    | 62.56%        | 58.94% (3)                        |
| 5.   | Maffra        | 59.28%        | 54.39% (8)                        |
| 6.   | Portland      | 58.00%        | 57.34% (6)                        |
| 7.   | Benella       | 57.23%        | 58.79% (5)                        |
| 8.   | Delacombe     | 57.23%        | 55.63% (7)                        |
| 9.   | Sale          | 52.56%        | 49.65% (11)                       |
| 10.  | Morwell       | 51.30%        | 53.89% (9)                        |


Building on this success, the model's forecast for the 2024-2028 period shows a continuation of similar trends, with strong growth expected in outer Melbourne suburbs. Notably, many of the top 10 forecasted high-growth suburbs are concentrated in the western regional areas of Melbourne, indicating a significant shift towards growth in these regions. This reinforces the reliability of the model in predicting future trends, particularly in identifying the top 10 growth suburbs, based on past performance.

<div style="display: flex; justify-content: space-between;">

<!-- Table on the left side -->
<div style="width: 45%;">
<h3>Predicted Growth for Suburbs:</h3>
<table>
  <tr>
    <th>Rank</th>
    <th>Suburb</th>
    <th>Predicted Growth</th>
  </tr>
  <tr><td>1.</td><td>Cairnlea</td><td>34.01%</td></tr>
  <tr><td>2.</td><td>Taylors Hill</td><td>33.77%</td></tr>
  
  <tr><td>3.</td><td>Weir Views</td><td>32.86%</td></tr>
  <tr><td>4.</td><td>Plenty</td><td>31.14%</td></tr>
  <tr><td>5.</td><td>Cobblebank</td><td>30.83%</td></tr>
  <tr><td>6.</td><td>Ballarat</td><td>29.61%</td></tr>
  <tr><td>7.</td><td>Taylors Lakes</td><td>28.71%</td></tr>
  <tr><td>8.</td><td>Kilsyth South</td><td>26.61%</td></tr>
  <tr><td>9.</td><td>Buninyong</td><td>22.26%</td></tr>
  <tr><td>10.</td><td>Brookfield</td><td>26.24%</td></tr>
</table>
</div>

<!-- Image on the right side -->
<div style="width: 45%;">
    <img src='plots/forecastgrowth.png' alt="Image Description" width="800">
</div>

</div>

#### 5.2 'What are the most liveable and affordable suburbs according to your chosen metrics?'

We adapted the Global Liveability Index from the Economist Intelligence Unit (EIU) and applied weights arbitrarily based on our discretion to relevant features that relate to stability, healthcare, culture and environment, education, and infrastructure and summed said features up to form an index of our own. A higher score would mean higher liveability and vice versa. Some of the biggest contributors to the score were crime, education, and beach proximity.

<img src='plots/Summary_liveability.png' alt="My Image" width="800"/>


We find that the top 10 most liveable suburbs such as Canterbury, Brighton, and Middle Park are centered around eastern melbourne while the least liveable suburbs such as Bendigo and Geelong are scattered outside of the city and regional victoria with the full list below along with their respective liveability scores. Interestingly enough a lot of the suburbs that are deemed least liveable are due to a high number of drug related crimes.

<div style="display: flex; justify-content: space-between;">

<div style="width: 45%;">
<h3>Top 10 Most Livable Suburbs:</h3>
<table>
  <tr>
    <th>Suburb Name</th>
    <th>Liveability Score</th>
  </tr>
  <tr><td>Canterbury (Vic.)</td><td>14.773933</td></tr>
  <tr><td>Brighton (Vic.)</td><td>13.177023</td></tr>
  <tr><td>Middle Park (Vic.)</td><td>12.863727</td></tr>
  <tr><td>Armadale (Vic.)</td><td>12.730818</td></tr>
  <tr><td>McKinnon</td><td>12.727558</td></tr>
  <tr><td>Hawthorn (Vic.)</td><td>12.672912</td></tr>
  <tr><td>Toorak</td><td>12.233129</td></tr>
  <tr><td>Glen Iris (Vic.)</td><td>12.088864</td></tr>
  <tr><td>Ormond</td><td>11.981344</td></tr>
  <tr><td>Caulfield North</td><td>11.308802</td></tr>
</table>
</div>

<div style="width: 45%;">
<h3>Bottom 10 Least Livable Suburbs:</h3>
<table>
  <tr>
    <th>Suburb Name</th>
    <th>Liveability Score</th>
  </tr>
  <tr><td>Sunshine (Vic.)</td><td>-23.082828</td></tr>
  <tr><td>Seymour (Vic.)</td><td>-23.650013</td></tr>
  <tr><td>Melton (Vic.)</td><td>-23.669923</td></tr>
  <tr><td>Bairnsdale</td><td>-24.691274</td></tr>
  <tr><td>Mildura</td><td>-27.376520</td></tr>
  <tr><td>Ballarat Central</td><td>-30.366898</td></tr>
  <tr><td>Morwell</td><td>-32.793555</td></tr>
  <tr><td>Caulfield East</td><td>-36.055177</td></tr>
  <tr><td>Geelong</td><td>-36.753450</td></tr>
  <tr><td>Bendigo</td><td>-39.830659</td></tr>
</table>
</div>

</div>


#### 5.3 Affordability

Affordability was calculated by taking the ratio of median family weekly income and adjusted average weekly rent. Based on that, we can see clearly that suburbs such as Southbank and Docklands near the CBD obviously have the lowest affordability as they command a higher weekly rent when compared to other suburbs while suburbs like Cobblebank and Aintree command a lower weekly rent which cause them to have a higher affordability in comparison. Another reason these suburbs more affordable suburbs could also be that they are apart of newer affordable housing developments. Another interesting point is that some of these more affordable subrubs also line up with the suburbs that have the highest growth based on our predictions.

<img src='plots/Summary_affordability_map.png' alt="My Image" width="800"/>


<div style="display: flex; justify-content: space-between;">

<div style="width: 45%;">
<h3>Top 10 Most Affordable Suburbs:</h3>
<table>
  <tr>
    <th>Suburb Name</th>
    <th>Affordability Score</th>
  </tr>
  <tr><td>Cobblebank</td><td>0.089716</td></tr>
  <tr><td>Aintree</td><td>0.088354</td></tr>
  <tr><td>Weir Views</td><td>0.084913</td></tr>
  <tr><td>Strathtulloh</td><td>0.084124</td></tr>
  <tr><td>Kalkallo</td><td>0.077302</td></tr>
  <tr><td>Truganina</td><td>0.077170</td></tr>
  <tr><td>Derrimut</td><td>0.075595</td></tr>
  <tr><td>Burnside Heights</td><td>0.074719</td></tr>
  <tr><td>Manor Lakes</td><td>0.074032</td></tr>
  <tr><td>Lysterfield South</td><td>0.074009</td></tr>
</table>
</div>

<div style="width: 45%;">
<h3>Bottom 10 Least Affordable Suburbs:</h3>
<table>
  <tr>
    <th>Suburb Name</th>
    <th>Affordability Score</th>
  </tr>
  <tr><td>Abbotsford (Vic.)</td><td>0.028743</td></tr>
  <tr><td>East Melbourne</td><td>0.028685</td></tr>
  <tr><td>South Melbourne</td><td>0.028651</td></tr>
  <tr><td>West Melbourne</td><td>0.028018</td></tr>
  <tr><td>Carlton (Vic.)</td><td>0.028002</td></tr>
  <tr><td>St Kilda West</td><td>0.027873</td></tr>
  <tr><td>South Yarra</td><td>0.027245</td></tr>
  <tr><td>Southbank</td><td>0.025384</td></tr>
  <tr><td>Docklands</td><td>0.023707</td></tr>
  <tr><td>Melbourne</td><td>0.021520</td></tr>
</table>
</div>

</div>


### 5.4 Summary of Challenges, Limitations & Assumptions for this Stage
Challenges
- How to adapt the liveability index to our liveability scoring metric

Limitations
- Data that contributed to the weightings of the liveability scoring categories: stability, healthcare, culture and environment, education, and infrastructure was limited to the data we scraped. This means we had more data on some categories then others, this still is believed not have a dramatic effect on results.

Assumptions
- The Global Liveability Index was an appropiate scoring metric to model our own weights off

## 6. Conclusion

Finally, based on our findings, we believe these insights can be transformed into valuable consulting services for various stakeholders, particularly online real estate firms. These firms are constantly seeking data-driven insights to inform smarter, strategic decisions, and we are confident they would be willing to pay a premium for services like ours. With our model, they can accurately determine rental prices and identify growth opportunities in their property portfolios. We could offer customized reports on rental price forecasts, growth potential, and livability insights, helping firms stay ahead of the market and capitalize on high-growth opportunities.

Given the progress we've made in just six weeks and the strength of our highly accurate model, this project is well worth pursuing. We could begin offering custom reports within 1-3 months and scale to a subscription-based service in 3-6 months by automating our processes. By partnering with real estate firms, we could also gain access to better data, further refining our model and enhancing the accuracy of our insights, creating a mutually beneficial opportunity for both us and our clients.
